<a href="https://colab.research.google.com/github/girinath18/CodeFormer/blob/Remastered/face_enhanacement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from IPython.display import clear_output
%cd /content
!rm -rf /content/CodeFormer
!git clone https://github.com/sczhou/CodeFormer.git
%cd /content/CodeFormer
!pip install -r /content/CodeFormer/requirements.txt
# Install basicsr
!python /content/CodeFormer/basicsr/setup.py develop
# Download the pre-trained model
!python /content/CodeFormer/scripts/download_pretrained_models.py facelib
!python /content/CodeFormer/scripts/download_pretrained_models.py CodeFormer
clear_output()

In [ ]:
from google.colab import drive
import os
drive.mount("/content/gdrive", force_remount=True)
drive_input_folder = "/content/gdrive/MyDrive/upload"
if os.path.exists(drive_input_folder):
    print("The folder exists.")
else:
    os.mkdir(drive_input_folder)
    print(f"Creating {drive_input_folder} folder")

Mounted at /content/gdrive
The folder exists.


In [ ]:
#@title Utils { display-mode: "form" }


# Visualization function
import cv2
import matplotlib.pyplot as plt
import os
import shutil
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('CodeFormer', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

upload_folder = ('/content/input')
if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.mkdir(upload_folder)
if os.path.exists("/content/output"):
  shutil.rmtree("/content/output")
clear_output()

In [2]:
from PIL import Image
import os
import shutil
from google.colab import drive
import os
from google.colab import files
keep_images = False
upload_folder = ('/content/input')
%cd /content/CodeFormer

if keep_images:
  pass
else:
  if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
  os.mkdir(upload_folder)


def image_from_drive():
  drive.mount('/content/gdrive',force_remount=True)
  drive_input_folder = "/content/input"
  if os.path.exists(drive_input_folder):
      print("The folder exists.")
  else:
      os.mkdir(drive_input_folder)
  image_extensions = ['.jpg', '.jpeg', '.png']
  for filename in os.listdir(drive_input_folder):
    _, extension = os.path.splitext(filename)
    if extension.lower() in image_extensions:
      drive_image_path=os.path.join(drive_input_folder,filename)
      shutil.copy(drive_image_path,upload_folder)
def image_from_device():
  uploaded = files.upload()
  for filename in uploaded.keys():
    dst_path = os.path.join(upload_folder, filename)
    print(f'move {filename} to {dst_path}')
    shutil.move(filename, dst_path)

upload_from = "local device"
if upload_from == "google drive":
  image_from_drive()
if upload_from == "local device":
  image_from_device()
clear_output()
file_count = len(os.listdir(upload_folder))
if file_count >= 1:
    print("Run next cell")
else:
    print("Please upload an image.")


/content/CodeFormer


KeyboardInterrupt: 

In [ ]:
# Inference the uploaded images
%cd /content/CodeFormer

CODEFORMER_FIDELITY = 0.7

BACKGROUND_ENHANCE = True

FACE_UPSAMPLE = True
if BACKGROUND_ENHANCE:
  if FACE_UPSAMPLE:
    !python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path inputs/user_upload --bg_upsampler realesrgan --face_upsample
  else:
    !python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path inputs/user_upload --bg_upsampler realesrgan
else:
  !python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path inputs/user_upload
clear_output()
print(f"All results are saved in /content/output{CODEFORMER_FIDELITY}")

All results are saved in /content/output0.7


In [ ]:

save_a_copy_in_google_drive = True
if save_a_copy_in_google_drive:
  drive.mount('/content/gdrive',force_remount=True)
  drive_save_path = '/content/gdrive/MyDrive/CodeFormer_Bulk_Upscale'
  if os.path.exists(drive_save_path):
    pass
  else:
    os.mkdir(drive_save_path)
  clear_output()
  print(f"All images/zip file save at : {drive_save_path}")

import os
import uuid
from google.colab import files
import shutil
def download_single_images():
  global CODEFORMER_FIDELITY
  download_folder = '/content/download'
  if os.path.exists(download_folder):
      os.system(f'rm -rf {download_folder}')
  os.makedirs(download_folder)
  folder_path = f"/content/CodeFormer/results/user_upload_{CODEFORMER_FIDELITY}/final_results"
  for filename in os.listdir(folder_path):
      original_path = os.path.join(folder_path, filename)
      name, extension = os.path.splitext(filename)
      random_string = str(uuid.uuid4())[:8]
      new_filename = f"{name}_{random_string}{extension}"
      download_path = f"/content/download/{new_filename}"
      if save_a_copy_in_google_drive:
        drive_path=f"{drive_save_path}/{new_filename}"
        shutil.copy(original_path, drive_path)
      shutil.copy(original_path, download_path)

      files.download(download_path)
def download_zip():
  global CODEFORMER_FIDELITY
  random_string = str(uuid.uuid4())[:5]
  zip_file_name=f"results_{random_string}.zip"
  var1=os.system(f'zip -r {zip_file_name} results/user_upload_{CODEFORMER_FIDELITY}/final_results')
  if save_a_copy_in_google_drive:
    sour=f"/content/CodeFormer/{zip_file_name}"
    dest=f"{drive_save_path}/{zip_file_name}"
    shutil.copy(sour, dest)

  files.download(zip_file_name)



folder_path = f"/content/CodeFormer/results/user_upload_{CODEFORMER_FIDELITY}/final_results"
image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']  # Add more extensions if needed
contains_image = False
for filename in os.listdir(folder_path):
    _, extension = os.path.splitext(filename)
    if extension.lower() in image_extensions:
        contains_image = True
        break
download_format = "zip"
  if download_format == "single file":
    download_single_images()
  if download_format == "zip":
    download_zip()
else:
  print(f"The folder '{folder_path}' does not contain any image files.")


IndentationError: unexpected indent (<ipython-input-5-10595a1c01dd>, line 58)

# **maded as the single script**

In [ ]:
# Required Libraries
import os
import shutil
import cv2
import matplotlib.pyplot as plt
from google.colab import files

# Visualization Function
def display(img1, img2):
    fig = plt.figure(figsize=(25, 10))
    ax1 = fig.add_subplot(1, 2, 1)
    plt.title('Input', fontsize=16)
    ax1.axis('off')
    ax2 = fig.add_subplot(1, 2, 2)
    plt.title('CodeFormer', fontsize=16)
    ax2.axis('off')
    ax1.imshow(img1)
    ax2.imshow(img2)

def imread(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"Image file not found at {img_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

# Paths
input_folder = '/content/input'
output_folder = '/content/output_bots'
codeformer_path = '/content/CodeFormer'

# Clean up previous runs
if os.path.isdir(input_folder):
    shutil.rmtree(input_folder)
os.mkdir(input_folder)
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.mkdir(output_folder)

# Image Upload
def image_from_device():
    uploaded = files.upload()
    for filename in uploaded.keys():
        dst_path = os.path.join(input_folder, filename)
        print(f'Move {filename} to {dst_path}')
        shutil.move(filename, dst_path)

# Execute Image Upload
image_from_device()
clear_output()
file_count = len(os.listdir(input_folder))
if file_count >= 1:
    print("Run next cell")
else:
    print("Please upload an image.")

# Inference the uploaded images
os.chdir(codeformer_path)

CODEFORMER_FIDELITY = 0.7
BACKGROUND_ENHANCE = True
FACE_UPSAMPLE = True

# Running the inference and saving outputs in specified output folder
command = f'python inference_codeformer.py -w {CODEFORMER_FIDELITY} --input_path {input_folder} --output_path {output_folder}'
if BACKGROUND_ENHANCE:
    command += ' --bg_upsampler realesrgan'
if FACE_UPSAMPLE:
    command += ' --face_upsample'

os.system(command)

# Debugging information to verify the output
print(f"All results are saved in {output_folder}")
output_files = os.listdir(output_folder)
print(f"Files in output folder: {output_files}")

# Check if output folder contains any images
if not output_files:
    raise FileNotFoundError(f"No output images found in {output_folder}")

# Display Results
input_image_path = os.path.join(input_folder, os.listdir(input_folder)[0])
output_image_path = os.path.join(output_folder, output_files[0])

print(f"Input image path: {input_image_path}")
print(f"Output image path: {output_image_path}")

input_image = imread(input_image_path)
output_image = imread(output_image_path)
display(input_image, output_image)


In [16]:
import os
import shutil
import cv2
import matplotlib.pyplot as plt

def process_and_display_images(input_folder):
    # Visualization Function
    def display(img1, img2):
        fig = plt.figure(figsize=(25, 10))
        ax1 = fig.add_subplot(1, 2, 1)
        plt.title('Input', fontsize=16)
        ax1.axis('off')
        ax2 = fig.add_subplot(1, 2, 2)
        plt.title('CodeFormer', fontsize=16)
        ax2.axis('off')
        ax1.imshow(img1)
        ax2.imshow(img2)

    def imread(img_path):
        img = cv2.imread(img_path)
        if img is None:
            raise FileNotFoundError(f"Image file not found at {img_path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img

    # Paths
    output_folder = '/content/output_bots/'
    codeformer_path = '/content/CodeFormer'

    # Clean up previous runs
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
    os.mkdir(output_folder)

    # Check if input folder contains any images
    if not os.path.isdir(input_folder) or not os.listdir(input_folder):
        print("The input folder is empty or does not exist. Please provide a valid input folder with images.")
        return

    # Inference the uploaded images
    os.chdir(codeformer_path)

    CODEFORMER_FIDELITY = 0.7
    BACKGROUND_ENHANCE = True
    FACE_UPSAMPLE = True

    # Running the inference and saving outputs in specified output folder
    command = f'python inference_codeformer.py -w {CODEFORMER_FIDELITY} --input_path {input_folder} --output_path {output_folder} --final_only'
    if BACKGROUND_ENHANCE:
        command += ' --bg_upsampler realesrgan'
    if FACE_UPSAMPLE:
        command += ' --face_upsample'

    print(f"Running command: {command}")
    return_code = os.system(command)

    if return_code != 0:
        print(f"Command failed with return code: {return_code}")
        print("Please check the command and try again.")
        return

    # Check if output folder contains any images
    output_files = os.listdir(output_folder)
    if not output_files:
        print("No output images found. Something went wrong.")
        return

    # Display Results
    input_image_path = os.path.join(input_folder, os.listdir(input_folder)[0])
    output_image_path = os.path.join(output_folder, output_files[0])

    input_image = imread(input_image_path)
    output_image = imread(output_image_path)
    display(input_image, output_image)

# Example usage: Call the function with the input folder path
input_folder_path = '/content/input/'
process_and_display_images(input_folder_path)


Running command: python inference_codeformer.py -w 0.7 --input_path /content/input/ --output_path /content/output_bots/ --final_only --bg_upsampler realesrgan --face_upsample
Command failed with return code: 512
Please check the command and try again.


# **inference_codeformer.py**

In [ ]:
import os
import cv2
import argparse
import glob
import torch
from torchvision.transforms.functional import normalize
from basicsr.utils import imwrite, img2tensor, tensor2img
from basicsr.utils.download_util import load_file_from_url
from basicsr.utils.misc import gpu_is_available, get_device
from facelib.utils.face_restoration_helper import FaceRestoreHelper
from facelib.utils.misc import is_gray

from basicsr.utils.registry import ARCH_REGISTRY

pretrain_model_url = {
    'restoration': 'https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/codeformer.pth',
}

def set_realesrgan():
    from basicsr.archs.rrdbnet_arch import RRDBNet
    from basicsr.utils.realesrgan_utils import RealESRGANer

    use_half = False
    if torch.cuda.is_available(): # set False in CPU/MPS mode
        no_half_gpu_list = ['1650', '1660'] # set False for GPUs that don't support f16
        if not True in [gpu in torch.cuda.get_device_name(0) for gpu in no_half_gpu_list]:
            use_half = True

    model = RRDBNet(
        num_in_ch=3,
        num_out_ch=3,
        num_feat=64,
        num_block=23,
        num_grow_ch=32,
        scale=2,
    )
    upsampler = RealESRGANer(
        scale=2,
        model_path="https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/RealESRGAN_x2plus.pth",
        model=model,
        tile=args.bg_tile,
        tile_pad=40,
        pre_pad=0,
        half=use_half
    )

    if not gpu_is_available():  # CPU
        import warnings
        warnings.warn('Running on CPU now! Make sure your PyTorch version matches your CUDA.'
                        'The unoptimized RealESRGAN is slow on CPU. '
                        'If you want to disable it, please remove `--bg_upsampler` and `--face_upsample` in command.',
                        category=RuntimeWarning)
    return upsampler

if __name__ == '__main__':
    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    device = get_device()
    parser = argparse.ArgumentParser()

    parser.add_argument('-i', '--input_path', type=str, default='./inputs/whole_imgs',
            help='Input image, video or folder. Default: inputs/whole_imgs')
    parser.add_argument('-o', '--output_path', type=str, default=None,
            help='Output folder. Default: results/<input_name>_<w>')
    parser.add_argument('-w', '--fidelity_weight', type=float, default=0.5,
            help='Balance the quality and fidelity. Default: 0.5')
    parser.add_argument('-s', '--upscale', type=int, default=2,
            help='The final upsampling scale of the image. Default: 2')
    parser.add_argument('--has_aligned', action='store_true', help='Input are cropped and aligned faces. Default: False')
    parser.add_argument('--only_center_face', action='store_true', help='Only restore the center face. Default: False')
    parser.add_argument('--draw_box', action='store_true', help='Draw the bounding box for the detected faces. Default: False')
    # large det_model: 'YOLOv5l', 'retinaface_resnet50'
    # small det_model: 'YOLOv5n', 'retinaface_mobile0.25'
    parser.add_argument('--detection_model', type=str, default='retinaface_resnet50',
            help='Face detector. Optional: retinaface_resnet50, retinaface_mobile0.25, YOLOv5l, YOLOv5n, dlib. \
                Default: retinaface_resnet50')
    parser.add_argument('--bg_upsampler', type=str, default='None', help='Background upsampler. Optional: realesrgan')
    parser.add_argument('--face_upsample', action='store_true', help='Face upsampler after enhancement. Default: False')
    parser.add_argument('--bg_tile', type=int, default=400, help='Tile size for background sampler. Default: 400')
    parser.add_argument('--suffix', type=str, default=None, help='Suffix of the restored faces. Default: None')
    parser.add_argument('--save_video_fps', type=float, default=None, help='Frame rate for saving video. Default: None')

    args = parser.parse_args()

    # ------------------------ input & output ------------------------
    w = args.fidelity_weight
    input_video = False
    if args.input_path.endswith(('jpg', 'jpeg', 'png', 'JPG', 'JPEG', 'PNG')): # input single img path
        input_img_list = [args.input_path]
        result_root = f'results/test_img_{w}'
    elif args.input_path.endswith(('mp4', 'mov', 'avi', 'MP4', 'MOV', 'AVI')): # input video path
        from basicsr.utils.video_util import VideoReader, VideoWriter
        input_img_list = []
        vidreader = VideoReader(args.input_path)
        image = vidreader.get_frame()
        while image is not None:
            input_img_list.append(image)
            image = vidreader.get_frame()
        audio = vidreader.get_audio()
        fps = vidreader.get_fps() if args.save_video_fps is None else args.save_video_fps
        video_name = os.path.basename(args.input_path)[:-4]
        result_root = f'results/{video_name}_{w}'
        input_video = True
        vidreader.close()
    else: # input img folder
        if args.input_path.endswith('/'):  # solve when path ends with /
            args.input_path = args.input_path[:-1]
        # scan all the jpg and png images
        input_img_list = sorted(glob.glob(os.path.join(args.input_path, '*.[jpJP][pnPN]*[gG]')))
        result_root = f'results/{os.path.basename(args.input_path)}_{w}'

    if not args.output_path is None: # set output path
        result_root = args.output_path

    test_img_num = len(input_img_list)
    if test_img_num == 0:
        raise FileNotFoundError('No input image/video is found...\n'
            '\tNote that --input_path for video should end with .mp4|.mov|.avi')

    # ------------------ set up background upsampler ------------------
    if args.bg_upsampler == 'realesrgan':
        bg_upsampler = set_realesrgan()
    else:
        bg_upsampler = None

    # ------------------ set up face upsampler ------------------
    if args.face_upsample:
        if bg_upsampler is not None:
            face_upsampler = bg_upsampler
        else:
            face_upsampler = set_realesrgan()
    else:
        face_upsampler = None

    # ------------------ set up CodeFormer restorer -------------------
    net = ARCH_REGISTRY.get('CodeFormer')(dim_embd=512, codebook_size=1024, n_head=8, n_layers=9,
                                            connect_list=['32', '64', '128', '256']).to(device)

    # ckpt_path = 'weights/CodeFormer/codeformer.pth'
    ckpt_path = load_file_from_url(url=pretrain_model_url['restoration'],
                                    model_dir='weights/CodeFormer', progress=True, file_name=None)
    checkpoint = torch.load(ckpt_path)['params_ema']
    net.load_state_dict(checkpoint)
    net.eval()

    # ------------------ set up FaceRestoreHelper -------------------
    # large det_model: 'YOLOv5l', 'retinaface_resnet50'
    # small det_model: 'YOLOv5n', 'retinaface_mobile0.25'
    if not args.has_aligned:
        print(f'Face detection model: {args.detection_model}')
    if bg_upsampler is not None:
        print(f'Background upsampling: True, Face upsampling: {args.face_upsample}')
    else:
        print(f'Background upsampling: False, Face upsampling: {args.face_upsample}')

    face_helper = FaceRestoreHelper(
        args.upscale,
        face_size=512,
        crop_ratio=(1, 1),
        det_model = args.detection_model,
        save_ext='png',
        use_parse=True,
        device=device)

    # -------------------- start to processing ---------------------
    for i, img_path in enumerate(input_img_list):
        # clean all the intermediate results to process the next image
        face_helper.clean_all()

        if isinstance(img_path, str):
            img_name = os.path.basename(img_path)
            basename, ext = os.path.splitext(img_name)
            print(f'[{i+1}/{test_img_num}] Processing: {img_name}')
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        else: # for video processing
            basename = str(i).zfill(6)
            img_name = f'{video_name}_{basename}' if input_video else basename
            print(f'[{i+1}/{test_img_num}] Processing: {img_name}')
            img = img_path

        if args.has_aligned:
            # the input faces are already cropped and aligned
            img = cv2.resize(img, (512, 512), interpolation=cv2.INTER_LINEAR)
            face_helper.is_gray = is_gray(img, threshold=10)
            if face_helper.is_gray:
                print('Grayscale input: True')
            face_helper.cropped_faces = [img]
        else:
            face_helper.read_image(img)
            # get face landmarks for each face
            num_det_faces = face_helper.get_face_landmarks_5(
                only_center_face=args.only_center_face, resize=640, eye_dist_threshold=5)
            print(f'\tdetect {num_det_faces} faces')
            # align and warp each face
            face_helper.align_warp_face()

        # face restoration for each cropped face
        for idx, cropped_face in enumerate(face_helper.cropped_faces):
            # prepare data
            cropped_face_t = img2tensor(cropped_face / 255., bgr2rgb=True, float32=True)
            normalize(cropped_face_t, (0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True)
            cropped_face_t = cropped_face_t.unsqueeze(0).to(device)

            try:
                with torch.no_grad():
                    output = net(cropped_face_t, w=w, adain=True)[0]
                    restored_face = tensor2img(output, rgb2bgr=True, min_max=(-1, 1))
                del output
                torch.cuda.empty_cache()
            except Exception as error:
                print(f'\tFailed inference for CodeFormer: {error}')
                restored_face = tensor2img(cropped_face_t, rgb2bgr=True, min_max=(-1, 1))

            restored_face = restored_face.astype('uint8')
            face_helper.add_restored_face(restored_face, cropped_face)

        # paste_back
        if not args.has_aligned:
            # upsample the background
            if bg_upsampler is not None:
                # Now only support RealESRGAN for upsampling background
                bg_img = bg_upsampler.enhance(img, outscale=args.upscale)[0]
            else:
                bg_img = None
            face_helper.get_inverse_affine(None)
            # paste each restored face to the input image
            if args.face_upsample and face_upsampler is not None:
                restored_img = face_helper.paste_faces_to_input_image(upsample_img=bg_img, draw_box=args.draw_box, face_upsampler=face_upsampler)
            else:
                restored_img = face_helper.paste_faces_to_input_image(upsample_img=bg_img, draw_box=args.draw_box)

        # save faces
        for idx, (cropped_face, restored_face) in enumerate(zip(face_helper.cropped_faces, face_helper.restored_faces)):
            # save cropped face
            if not args.has_aligned:
                save_crop_path = os.path.join(result_root, 'cropped_faces', f'{basename}_{idx:02d}.png')
                imwrite(cropped_face, save_crop_path)
            # save restored face
            if args.has_aligned:
                save_face_name = f'{basename}.png'
            else:
                save_face_name = f'{basename}_{idx:02d}.png'
            if args.suffix is not None:
                save_face_name = f'{save_face_name[:-4]}_{args.suffix}.png'
            save_restore_path = os.path.join(result_root, 'restored_faces', save_face_name)
            imwrite(restored_face, save_restore_path)

        # save restored img
        if not args.has_aligned and restored_img is not None:
            if args.suffix is not None:
                basename = f'{basename}_{args.suffix}'
            save_restore_path = os.path.join(result_root, 'final_results', f'{basename}.png')
            imwrite(restored_img, save_restore_path)

    # save enhanced video
    if input_video:
        print('Video Saving...')
        # load images
        video_frames = []
        img_list = sorted(glob.glob(os.path.join(result_root, 'final_results', '*.[jp][pn]g')))
        for img_path in img_list:
            img = cv2.imread(img_path)
            video_frames.append(img)
        # write images to video
        height, width = video_frames[0].shape[:2]
        if args.suffix is not None:
            video_name = f'{video_name}_{args.suffix}.png'
        save_restore_path = os.path.join(result_root, f'{video_name}.mp4')
        vidwriter = VideoWriter(save_restore_path, height, width, fps, audio)

        for f in video_frames:
            vidwriter.write_frame(f)
        vidwriter.close()

    print(f'\nAll results are saved in {result_root}')


In [ ]:
# Required Libraries
import os
import shutil
import cv2
import matplotlib.pyplot as plt
from google.colab import files

# Visualization Function
def display(img1, img2):
    fig = plt.figure(figsize=(25, 10))
    ax1 = fig.add_subplot(1, 2, 1)
    plt.title('Input', fontsize=16)
    ax1.axis('off')
    ax2 = fig.add_subplot(1, 2, 2)
    plt.title('CodeFormer', fontsize=16)
    ax2.axis('off')
    ax1.imshow(img1)
    ax2.imshow(img2)

def imread(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"Image file not found at {img_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

# Paths
input_folder = '/content/input'
output_folder = '/content/output_bots'
codeformer_path = '/content/CodeFormer'

# Clean up previous runs
if os.path.isdir(input_folder):
    shutil.rmtree(input_folder)
os.mkdir(input_folder)
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.mkdir(output_folder)

# Image Upload
def image_from_device():
    uploaded = files.upload()
    for filename in uploaded.keys():
        dst_path = os.path.join(input_folder, filename)
        print(f'Move {filename} to {dst_path}')
        shutil.move(filename, dst_path)

# Execute Image Upload
image_from_device()
clear_output()
file_count = len(os.listdir(input_folder))
if file_count >= 1:
    print("Run next cell")
else:
    print("Please upload an image.")

# Inference the uploaded images
os.chdir(codeformer_path)

CODEFORMER_FIDELITY = 0.7
BACKGROUND_ENHANCE = True
FACE_UPSAMPLE = True

# Running the inference and saving outputs in specified output folder
command = f'python inference_codeformer.py -w {CODEFORMER_FIDELITY} --input_path {input_folder} --output_path {output_folder}'
if BACKGROUND_ENHANCE:
    command += ' --bg_upsampler realesrgan'
if FACE_UPSAMPLE:
    command += ' --face_upsample'

os.system(command)

# Debugging information to verify the output
print(f"All results are saved in {output_folder}")
output_files = os.listdir(output_folder)
print(f"Files in output folder: {output_files}")

# Check if output folder contains any images
final_results_folder = os.path.join(output_folder, 'final_results')
if not os.path.exists(final_results_folder):
    raise FileNotFoundError(f"No final_results folder found in {output_folder}")

final_results_files = os.listdir(final_results_folder)
if not final_results_files:
    raise FileNotFoundError(f"No output images found in {final_results_folder}")

# Display Results
input_image_path = os.path.join(input_folder, os.listdir(input_folder)[0])
output_image_path = os.path.join(final_results_folder, final_results_files[0])

print(f"Input image path: {input_image_path}")
print(f"Output image path: {output_image_path}")

input_image = imread(input_image_path)
output_image = imread(output_image_path)
display(input_image, output_image)
